In [1]:
import pandas as pd
import sqlalchemy as sql
import pyodbc as pyo
import numpy as np

In [2]:
server = 'CALUMO-949324'
driver = 'SQL+Server'
db = 'Calumo'
myQuery = '''SELECT *from Users'''
engine = sql.create_engine('mssql+pyodbc://{}/{}?driver={}'.format(server, db, driver))
Users = pd.read_sql_query(myQuery, engine)

In [3]:
myQuery2 = '''SELECT *from ReportPerformanceEvent'''

In [4]:
RPF = pd.read_sql_query(myQuery2, engine)

In [5]:
myQuery3 = 'Select * from UserGroups'

In [6]:
UG = pd.read_sql_query(myQuery3, engine)

In [7]:
intrim = Users.merge(RPF,left_on='LoginID', right_on='UserName')


In [8]:
pd.set_option('display.max_columns', 50)

In [9]:
df =intrim.merge(UG,left_on='Id_x', right_on='UserID').rename(columns={'LoginID':'userLogin','Id_x':'UserKey'})


In [10]:
Name = df['FirstName'] +" " + df['LastName']

In [11]:
df.insert(1,"Name",Name)

In [12]:
CurrentUser = df['IsDeleted'] == 0

In [13]:
CurrentUser.value_counts()

True     63489
False    11656
Name: IsDeleted, dtype: int64

In [16]:
df = df[(df['IsDeleted'] == 0) & (df['Name'].notna())]

In [17]:
df.groupby (['Name', 'Email','UserKey']).agg({'Id_y':np.size}).rename(columns={'Id_y':'UsageCount'})

,,,UsageCount
Name,Email,UserKey,
Casey Centenera,casey.centenera@artscentremelbourne.com.au,9,10005
Chris Kim,ckim@calumo.com,23,4
David Elsey,delsey@calumo.com,4,169
Geoffrey Gillian,ggillian@calumo.com,19,270
James Lockyer,james.lockyer@artscentremelbourne.com.au,10,273
James Summerton,jsummerton@calumo.com,1,13
Jamie McMillan,jamie.mcmillan@artscentremelbourne.com.au,12,10952
Jessie Greig,jessie.greig@artscentremelbourne.com.au,14,14056
Kate Tayler,kate.tayler@artscentremelbourne.com.au,7,3612


SQL

BEGIN
SELECT 
U.FirstName + ' ' + U.LastName as UserName
,' ' + ISNULL(CONVERT(CHAR(11), CONVERT(datetime, SWITCHOFFSET(MIN(EventTime), DATEPART(TZOFFSET, 
		MIN(EventTime) AT TIME ZONE 'AUS Eastern Standard Time'))), 113),'-') as FirstUsedTime
,' ' + ISNULL(CONVERT(CHAR(11), CONVERT(datetime, SWITCHOFFSET(MAX(EventTime), DATEPART(TZOFFSET, 
		MAX(EventTime) AT TIME ZONE 'AUS Eastern Standard Time'))), 113),'-') as LastUsedTime
,COUNT(rpe.Id) as UsageCount
,u.LoginID as userLogin
,u.Email as userEmail
,u.Id as userKey
FROM Users u with (nolock) 
LEFT JOIN ReportPerformanceEvent  rpe with (nolock) on u.LoginID = rpe.UserName
WHERE UserName IS NOT NULL
 AND u.IsDeleted = 0
GROUP BY UserName, u.isDeleted, u.LoginID, u.Email, u.Id, u.FirstName, u.LastName
ORDER BY UserName
END
GO